In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm._tqdm import trange

In [ ]:
age_train = pd.read_csv("./input/age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv("./input/age_test.csv", names=['uid'])
app_info = pd.read_csv("./input/app_info.csv", names=['appId', 'category'])

In [ ]:
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()
    return np.nan if len(s) == 0 else s.index[0]
def processUserAppUsage():
    resTable = pd.DataFrame()
    date_use = pd.DataFrame()
    reader = pd.read_csv("./input/user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    
    i = 1
    
    while True:
        try:
            print("index: {}".format(i))
            i+=1
            df = reader.get_chunk(50000000)
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]
            
            
            now_df = pd.DataFrame()
            now_df['uid'] = df['uid'].unique()
            now_df = now_df.merge(df.groupby('uid')['appId'].count().to_frame(), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['appId','use_date'].agg(['nunique']), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']), how='left', on='uid')
            now_df.columns = ['uid','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']

            df.drop(['appId'], axis=1, inplace=True)
            allsum = df.groupby(['uid', 'use_date'])['duration','times'].sum().reset_index()
            allsum.drop('use_date', axis=1), inplace=True
            tmp = allsum.groupby('uid').agg(['min', 'max', 'mean', 'std'])
            tmp.columns = ['uid', 'duration_min', 'duration_max', 'duration_mean', 'duration_std', 'times_min', 'times_max', 'times_mean', 'times_std']
            
            now_df = now_df.merge(tmp, how='left', on='uid')
            
            resTable = pd.concat([resTable, now_df])
            
        except StopIteration:
            break
    
    resTable.to_csv("secondVersion.csv",index=0)
    
    print("Iterator is stopped")

In [ ]:
processUserAppUsage()